In [19]:
#Dependencies
from citipy import citipy

import os
import random
import math

import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn

In [20]:
# API Info
from config import key

base_url = "http://api.openweathermap.org/data/2.5/weather?appid="

In [40]:
#Generate a random set of 500 latitude and longitude values
rand_lat = []
rand_long = []

for i in range(600):
    rand_long.append(random.uniform(180,-180))
    
    lat = math.degrees(math.acos(2*random.random()-1))-90
    rand_lat.append(lat)

print(rand_long[0:5])
print(rand_lat[0:5])


[-40.6832681859882, 13.53591741764535, 110.54266766320717, -33.60768234798013, -95.86267324115744]
[15.742650231265216, -10.353712768254866, -83.32193435552932, -31.78714388650254, -28.68769886467564]


In [41]:
#initiate city, country lists to be populated within the for loop
cities = []
countries = []
loc_concat_list = []

#find the closest city through each of the 500 random lat/longs

for indexi, i in enumerate(rand_lat):
    
    citi = citipy.nearest_city(i, rand_long[indexi])
    
    cityname =citi.city_name
    countrycd = citi.country_code
    
    loc_concat = cityname + "," + countrycd
    
#Using a while loop: check to see if the location has already been chosen
#If it has, keep generating new lat/longs and city/countries until a unique location is found
    while loc_concat in loc_concat_list:
        
        rand_long[indexi] = random.uniform(180,-180)
        rand_lat[indexi] = math.degrees(math.acos(2*random.random()-1))-90
        
        citi = citipy.nearest_city(rand_lat[indexi], rand_long[indexi])
        
        cityname =citi.city_name
        countrycd = citi.country_code
        
        loc_concat = cityname + "," + countrycd
        #print(loc_concat)
    
#add the randomly generated cities and counties to lists   
    cities.append(cityname)
    countries.append(countrycd)
    loc_concat_list.append(cityname + "," + countrycd)

#print(cities[0:5])
print(loc_concat_list[0:5])

['ponta do sol,cv', 'sumbe,ao', 'albany,au', 'arraial do cabo,br', 'lebu,cl']


In [42]:
#create a dataframe of cities and countries
#write the information to a csv

cities_data = pd.DataFrame({"city": cities,"country": countries,"loc_concat":loc_concat_list,"latitude":rand_lat,"longitude":rand_long})

cities_data.to_csv(path_or_buf="check.csv")

In [43]:
#initiate lists to hold api results
api_lon = []
api_lat = []
api_temp = []
api_humid = []
api_cloudiness = []
api_windspeed = []

#loop through all cities, and search/return information from openweather API
for indexi, item in enumerate(loc_concat_list[0:16]):
    if len(api_lon) == 500:
        break
    q_url = base_url + key + "&q=" +item
    weather_response = requests.get(q_url)
    weather_json = weather_response.json()
    print("Processing city: " + str(indexi) +", "+ str(cities[indexi]) +" using " + str(q_url))
    
    #place each resulting value in its own list
    try:
        api_lon.append(weather_json["coord"]["lon"])
        api_lat.append(weather_json["coord"]["lat"])
        api_temp.append(weather_json["main"]["temp"]) #need to convert to farenheit
        api_humid.append(weather_json["main"]["humidity"])
        api_cloudiness.append(weather_json["clouds"]["all"]) #check units
        api_windspeed.append(weather_json["wind"]["speed"]) #check units

#Temperature (F) vs. Latitude
#Humidity (%) vs. Latitude
#Cloudiness (%) vs. Latitude
#Wind Speed (mph) vs. Latitude
    except:
        print(cities[indexi] + " cannot be found. Continuing with next city")
        continue
print(api_lon)

Processing city: 0, ponta do sol using http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&q=ponta do sol,cv
Processing city: 1, sumbe using http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&q=sumbe,ao
Processing city: 2, albany using http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&q=albany,au
Processing city: 3, arraial do cabo using http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&q=arraial do cabo,br
Processing city: 4, lebu using http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&q=lebu,cl
Processing city: 5, rikitea using http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&q=rikitea,pf
Processing city: 6, khonuu using http://api.openweathermap.org/data/2.5/weather?appid=25bc90a1196e6f153eece0bc0b0fc9eb&q=khonuu,ru
khonuu cannot be found. Continuing with next city
Process